In [2]:
import pandas as pd
import re
import openpyxl

# "保险公司":".+保险((?!资产管理).)*(股份有限|有限责任|有限|)公司$|.+保险社$"

#所有正则表达式
dict_regex={"企业集团财务公司":".+财务.*(股份有限|有限责任|有限|)公司$", \
            "村镇银行":".+村镇银行.*(股份有限|有限责任|有限|)公司$", \
            "公募证券投资基金管理公司":".+基金.*(股份有限|有限责任|有限|)公司$", \
            "金融资产投资公司":".+金融资产投资.*(股份有限|有限责任|有限|)公司$", \
            "金融租赁公司":".+金融租赁.*(股份有限|有限责任|有限|)公司$", \
            "农村商业银行":".+农村商业银行(股份有限|有限责任)公司$|.+农村合作银行$", \
            "农村信用社":".+(农村信用合作(联|)社|农村信用(合作|)社联合社|农村信用(合作|)联社股份有限公司|农村合作金融结算服务中心)$", \
            "期货公司":".+期货.*(股份有限|有限责任|有限|)公司$", \
            "银行间市场二级托管总账户":".+公司托管总账户$", \
            "外资法人银行":"^.+银行.*(\(|（)中国(\)|）)有限公司$", \
            "信托公司":".+信托.*(股份有限|有限责任|有限|)公司$", \
            "银行理财子公司":".+理财.*(股份有限|有限责任|有限|)公司$", \
            "证券公司":"^.+证券((?!资产管理).)*(股份有限|有限责任|有限|)公司$", \
            "保险保障基金":".*保险保障基金.+组合$", \
            "职业年金":".+职业年金.+$", \
            "企业年金":".+企业年金.+$", \
            "基本养老保险基金":".*基本养老保险基金.+组合$", \
            "全国社会保障基金":".+社保基金.+组合$", \
            "养老金产品":".+养老金(产品|计划).*$", \
            "公开募集证券投资基金":"^((?!私募).)+(证券投资基金.*|基金中基金.*|货币市场基金|投资基金|基金)$", \
            "私募投资基金":"^.+(私募证券投资基金.*|证券投资私募基金|私募基金.*|基金)$", \
            "银行理财产品":"^$"} #有些银行理财产品被分成法人机构，为了跑通程序作此设置

#需专门维护类型
special=["保险公司（停用）","保险资产管理公司","股份制商业银行","国有大型商业银行","民营银行","城市商业银行","其他基金管理公司", \
         "其他未列明银行业存款类金融机构","其他未列明银行业非存款类金融机构","其他证券业金融机构","商业银行分支机构（停用）", \
         "事业单位（停用）","特批非金融机构","特批机构","境外人民币清算行", "外国及港澳台银行境内分行","政策性银行","政府部门（停用）", \
         "住房储蓄银行","金融资产管理公司","信托业保障基金"]

#不容易区分
not_detectable=["保险产品独立账户","保险资管产品"]

#法人，只用正则表达式就可区分
is_regex_only=["企业集团财务公司","村镇银行","公募证券投资基金管理公司","金融资产投资公司","金融租赁公司","农村商业银行","农村信用社", \
               "期货公司","银行间市场二级托管总账户","外资法人银行","信托公司","银行理财子公司", "证券公司"]

#非法人，只用正则表达式就可区分
isnot_regex_only=["保险保障基金","职业年金","企业年金","基本养老保险基金","全国社会保障基金","养老金产品"]

#非法人，只用管理人信息就可区分
isnot_manager_only=["信托公司资管产品","银行理财产品","期货公司及其子公司资管产品","证券公司及其子公司资管产品", \
                    "基金管理公司及其子公司专户"]

#非法人，用正则表达式和管理人信息可区分
isnot_regex_manager=["公开募集证券投资基金","私募投资基金"]

#管理人名单字典
dict_manager={}
for m in isnot_manager_only+isnot_regex_manager:
    df_manager=pd.read_excel("./管理人名单.xlsx",sheet_name="{}".format(m))
    dict_manager[m]=list(df_manager["投资/资产管理人"])

#需单独维护名单
df_special=pd.read_excel("./需单独维护名单.xlsx")

#主表，统一列标题
df=pd.read_excel("./机构类别.xlsx")
if '持有人账号' in df.columns:
    df.rename(columns={'持有人账号':'账户账号','持有人中文全称':'账户全称','机构类别':'账户类型','非法人机构对应的法人机构':'投资/资产管理人'},inplace=True)
wb=openpyxl.load_workbook("./机构类别.xlsx")
ws=wb["Sheet1"]

#对每一条账户数据进行判定
for i in range(len(df)):
    
    #1类别在单独维护范围内
    if df.loc[i,"账户类型"] in special:
        
        #在需单独维护名单中寻找
        for j in range(len(df_special)):
            if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
    
    #2法人机构
    elif df.loc[i,"是否法人机构"]=="是":
        
        #根据自身类别判断正则表达式
        if re.match(dict_regex[df.loc[i,"账户类型"]],df.loc[i,"账户全称"],flags=0):
            ws["F{}".format(i+2)].value=df.loc[i,"账户类型"]
            ws["G{}".format(i+2)].value="是"
        else:
            flag=True
            
            #在需单独维护名单中寻找
            for j in range(len(df_special)):
                if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                    flag=False
                    ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                    ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
            if flag:
                
                #对其余正则表达式依次判定
                for key in is_regex_only:
                    if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0):
                        ws["F{}".format(i+2)].value=key
                        ws["G{}".format(i+2)].value="不一致"
    #3非法人产品
    else:
        
        #3.1非保险类产品
        if df.loc[i,"账户类型"] not in not_detectable:
            
            #3.1.1根据自身类别判断，可直接用正则表达式
            if df.loc[i,"账户类型"] in isnot_regex_only:
                if re.match(dict_regex[df.loc[i,"账户类型"]],df.loc[i,"账户全称"],flags=0):
                    ws["F{}".format(i+2)].value=df.loc[i,"账户类型"]
                    ws["G{}".format(i+2)].value="是"
                else:
                    flag=True
                    
                    #依次根据需单独维护名单、非法人直接正则表达式、非法人正则表达式管理人信息、非法人直接管理人信息判定，若都不是，判定为保险类产品
                    for j in range(len(df_special)):
                        if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                            flag=False
                            ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                            ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
                    if flag:
                        for key in isnot_regex_only:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0):
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_regex_manager:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0) and df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_manager_only:
                            if df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        ws["F{}".format(i+2)].value="保险类产品"
                        ws["G{}".format(i+2)].value="不一致"
            
            #3.1.2根据自身类别判断，用正则表达式&管理人信息
            elif df.loc[i,"账户类型"] in isnot_regex_manager:
                if re.match(dict_regex[df.loc[i,"账户类型"]],df.loc[i,"账户全称"],flags=0) and df.loc[i,"投资/资产管理人"] in dict_manager[df.loc[i,"账户类型"]]:
                    ws["F{}".format(i+2)].value=df.loc[i,"账户类型"]
                    ws["G{}".format(i+2)].value="是"
                else:
                    flag=True
                    
                    #依次根据需单独维护名单、非法人直接正则表达式、非法人正则表达式管理人信息、非法人直接管理人信息判定，若都不是，判定为保险类产品
                    for j in range(len(df_special)):
                        if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                            flag=False
                            ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                            ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
                    if flag:
                        for key in isnot_regex_only:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0):
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_regex_manager:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0) and df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_manager_only:
                            if df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        ws["F{}".format(i+2)].value="保险类产品"
                        ws["G{}".format(i+2)].value="不一致"
            
            #3.1.3根据自身类别判断，可直接用管理人信息
            else:
                if df.loc[i,"投资/资产管理人"] in dict_manager[df.loc[i,"账户类型"]]:
                    ws["F{}".format(i+2)].value=df.loc[i,"账户类型"]
                    ws["G{}".format(i+2)].value="是"
                else:
                    flag=True
                    
                    #依次根据需单独维护名单、非法人直接正则表达式、非法人正则表达式管理人信息、非法人直接管理人信息判定，若都不是，判定为保险类产品
                    for j in range(len(df_special)):
                        if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                            flag=False
                            ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                            ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
                    if flag:
                        for key in isnot_regex_only:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0):
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_regex_manager:
                            if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0) and df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        for key in isnot_manager_only:
                            if df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                                flag=False
                                ws["F{}".format(i+2)].value=key
                                ws["G{}".format(i+2)].value="不一致"
                    if flag:
                        ws["F{}".format(i+2)].value="保险类产品"
                        ws["G{}".format(i+2)].value="不一致"
        
        #3.2保险类产品
        else:
            flag=True
            
            #依次根据需单独维护名单、非法人直接正则表达式、非法人正则表达式管理人信息、非法人直接管理人信息判定，若都不是，判定为保险类产品
            for j in range(len(df_special)):
                if df_special.loc[j,"账户账号"]==df.loc[i,"账户账号"]:
                    flag=False
                    ws["F{}".format(i+2)].value=df_special.loc[j,"账户类型"]
                    ws["G{}".format(i+2)].value=df.loc[i,"账户类型"]==df_special.loc[j,"账户类型"]
            if flag:
                for key in isnot_regex_only:
                    if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0):
                        flag=False
                        ws["F{}".format(i+2)].value=key
                        ws["G{}".format(i+2)].value="不一致"
            if flag:
                for key in isnot_regex_manager:
                    if re.match(dict_regex[key],df.loc[i,"账户全称"],flags=0) and df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                        flag=False
                        ws["F{}".format(i+2)].value=key
                        ws["G{}".format(i+2)].value="不一致"
            if flag:
                for key in isnot_manager_only:
                    if df.loc[i,"投资/资产管理人"] in dict_manager[key]:
                        flag=False
                        ws["F{}".format(i+2)].value=key
                        ws["G{}".format(i+2)].value="不一致"
            if flag:
                ws["F{}".format(i+2)].value="保险类产品"
                ws["G{}".format(i+2)].value="一致"
                
wb.save("./机构类别.xlsx")